In [1]:
import os
import numpy as np
import gym
import tensorflow as tf
from time import time
from GA.genetic_algorithm import GeneticNetworks

from utils.atari_environment import AtariEnvironment
from utils.continuous_environments import Environment

from keras.backend.tensorflow_backend import set_session
from utils.networks import get_session

Using TensorFlow backend.


In [2]:
gym.logger.set_level(40)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
tensorboard_logs = "GA/tensorboard_gym"

In [4]:
# Clear any logs from previous runs
import shutil
shutil.rmtree(tensorboard_logs)

In [5]:
set_session(get_session())
summary_writer = tf.summary.FileWriter(tensorboard_logs)

In [6]:
env_name = 'CartPole-v0'
# env_name = 'Acrobot-v1'
# env_name = 'MountainCar-v0'
args = {
    'gym_env': gym.make(env_name),
    'action_repeat': 4
}
env = Environment(**args)
print('observation_space ', env.get_state_size())
print('action_space ', gym.make(env_name).action_space)

action_dim = gym.make(env_name).action_space.n
space_dim = env.get_state_size()

print(action_dim, space_dim, env_name)

observation_space  (4,)
action_space  Discrete(2)
2 (4,) CartPole-v0


In [13]:
genetic_pop = GeneticNetworks(architecture=(4,16,action_dim),
                                population_size=64, 
                                generations=10,
                                episodes=15, 
                                mutation_variance=0.1,
                                render_env=False,
                                verbose=True)

--------------------------------------------------
Logging params: 
population_size 64
generations 10
mutation_variance 0.1
survival_ratio 0.1
both_parent_percentage 0.8
one_parent_percentage 0.1
episodes 15
--------------------------------------------------


## Train

In [12]:
%%time
genetic_pop.fit(env, summary_writer, num_cpus=1, env_args=args)
print('Finished in',round(time()-start_time,3),'seconds')



Generation:   0%|          | 0/10 [00:00<?, ? gen/s]

ValueError: could not broadcast input array from shape (16,16) into shape (16)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame(genetic_pop.global_info)
print(df)

fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.plot(df['max'], label='max')
plt.plot(df['mean'], label='mean')
plt.legend()
plt.show()

## Evaluate

In [10]:
# Lets observe our best network
genetic_pop.best_network.evaluate(env, episodes=3, render_env=False, record=False)

181.33333333333334